# Intermediate Representation

Miasm provides an [intermediate representation](https://en.wikipedia.org/wiki/Intermediate_representation)
(*IR*) to represent the effects of a source code (for example, like LLVM). The benefits of using an IR are:
* A unified representation that does not depend on the source architecture;
* A minimal language;
* The side effects are explicit, for example *A + B* will not implicitly update flags.

Miasm's IR implementation is located in `miasm.expression.expression`, with `Expr*` objects. Each object has a size, in bits.

## Language

The basic words in the language are:
* `ExprId` : represents an identifier. For example, the register `EAX` will be represented by an `ExprId` of size 32 bits.
* `ExprInt` : represents an unsigned integer.

In [1]:
from miasm.expression.expression import *

a = ExprId("a", 32)
print(a)
print(repr(a))

# Identifier
print(a.name)
print(a.size)

a
ExprId('a', 32)
a
32


In [2]:
cst1 = ExprInt(16, 32)
print(cst1)
cst2 = ExprInt(-1, 32)
print(cst2)

# Show associated value
print(int(cst1))

0x10
0xFFFFFFFF
16


The word `ExprMem` represents a memory access, of a given size in bits.

In [3]:
# Memory access of 16 bits, at address 0x11223344 on 32 bits
addr = ExprInt(0x11223344, 32)
mem1 = ExprMem(addr, 16)
print(mem1)

# Show memory address
print(mem1.ptr)

@16[0x11223344]
0x11223344


The word `ExprOp` describes the n-ary operations between expressions. The operation is a string, so new operations can be created on the fly. Some operations (`+`, `*`, `|`, `parity`, ...) are already used by Miasm. An operation occurs between elements having the same size and has the same size as the arguments.

In [4]:
# Defining an operation
op1 = ExprOp("+", a, cst1)
print(op1)

# Accessing the arguments
print(op1.args)

# Creating a custom operation
op2 = ExprOp("MyCustomOp", a)
print(op2)

a + 0x10
(ExprId('a', 32), ExprInt(0x10, 32))
MyCustomOp(a)


Helpers ease the creation of common operations.

In [5]:
print(a + cst1)
print(a * cst1)
print(- a)
print(a | cst1)
print(a & cst1)

a + 0x10
a * 0x10
-a
a | 0x10
a & 0x10


Be careful, even though the Expressions can represent "everything", Miasm assumes some properties on certain operations:
* the associative operations (`+`, `^`, `|`, ...) are n-ary operations ;
* the `-` is always unary

In [6]:
print(a - cst1)

a + -0x10


* `parity` has always a size 1, it's an exception

In [7]:
p = ExprOp("parity", a)
print(a.size)
print(p.size)

32
1


The `=` operation is handled separately by the word `ExprAssign`.

In [8]:
assign = ExprAssign(a, cst1)
print(assign)

# Source, destination
print(assign.src)
print(assign.dst)

a = 0x10
0x10
a


The word `ExprCond` represents a ternary relation, equivalend to the Python `src1 if cond else src2`

In [9]:
cond = ExprCond(a, cst1, cst2)
print(cond)

# Access to the elements
print(cond.cond)
print(cond.src1)
print(cond.src2)

a?(0x10,0xFFFFFFFF)
a
0x10
0xFFFFFFFF


The following words manipulate the sizes:
* `ExprSlice`: extracts a bits slice of an expression;
* `ExprCompose`: concatenates two expressions.

In [10]:
sl = ExprSlice(a, 6, 8)
print(sl)
print(sl.size)

# Access to the properties
print(sl.arg)
print(sl.start)
print(sl.stop)

# Simpler form
sl == a[6:8]

a[6:8]
2
a
6
8


True

In [11]:
# Concatenation of a (bit 0 to 31) with cst1 (bit 32 to 63)
comp = ExprCompose(a, cst1)
print(comp)
print(comp.size)

# Access to the arguments
print(comp.args)
# Access to the starting bit and the associated argument
print(list(comp.iter_args()))

{a 0 32, 0x10 32 64}
64
(ExprId('a', 32), ExprInt(0x10, 32))
[(0, ExprId('a', 32)), (32, ExprInt(0x10, 32))]


**Rule: The size of an ```ExprCompose``` is equal to the sum of the sizes of its arguments.**

Finally, the word `ExprLoc` represents a memory location.
For example, it can represent the destination of a jump or a function call.

A location is described by a unique element of type  `LocKey`. You can see the `LocKey` as a key that you can use to retrieve all the information associated with a location: its offset, its name ("main") etc.
`ExprLoc` is a kind of `LocKey` container.

In [12]:
loc = ExprLoc(LocKey(1), 32)
print(loc)

loc_key_1


In summary, the different words are:

| Word | Meaning |
|-----|----------|
|ExprAssign|A=B|
|ExprInt|0x18|
|ExprId|EAX|
|ExprLoc|label_1|
|ExprCond|A ? B : C|
|ExprMem|@16[ESI]|
|ExprOp|A + B|
|ExprSlice|AH = EAX[8 :16]|
|ExprCompose|AX = AH.AL|

## Common helpers

In [13]:
# Proper size mask
a.mask

ExprInt(0xFFFFFFFF, 32)

In [14]:
# Expression size
a.size

32

In [15]:
# Printable version
print(a, cst1)

a 0x10


In [16]:
# Expr representation (can be copy-pasted in the code)
print(repr(a), repr(a + cst1))

ExprId('a', 32) ExprOp('+', ExprId('a', 32), ExprInt(0x10, 32))


In [17]:
# Size extension (unsigned, signed)
print(cst1.zeroExtend(64))
print(cst1.signExtend(64))

zeroExt_64(0x10)
signExt_64(0x10)


In [18]:
# Most significant bit
print(a.msb())

a[31:32]


In [19]:
# Replacement
expr1 = a + a + cst1
print(expr1)
expr2 = expr1.replace_expr({a: cst2})
print(expr2)

a + a + 0x10
0xFFFFFFFF + 0xFFFFFFFF + 0x10


In [20]:
# Type test
print(a.is_id())
print(a.is_int())
print(cst1.is_int())
print(op1.is_op())
print(op1.is_op("+"))
print(op1.is_op("&"))

True
False
True
True
True
False


## Expression represented by a graph

Miasm IR expressions have a recursive structure and can be represented and manipulated as graphs.
The graph object is a `DiGraph`, implemented in `miasm.core.graph`. It offers usual methods for manipulating graphs (node and vertex access, predecessors, successors, dominators, graphviz dot representation ...).

In [21]:
expr3 = a + cst1 & cst2
print(expr3)
graph = expr3.graph()
print(graph)

(a + 0x10) & 0xFFFFFFFF
0xFFFFFFFF
a
a + 0x10
(a + 0x10) & 0xFFFFFFFF
0x10
a + 0x10 -> a
a + 0x10 -> 0x10
(a + 0x10) & 0xFFFFFFFF -> a + 0x10
(a + 0x10) & 0xFFFFFFFF -> 0xFFFFFFFF


In [22]:
dot = graph.dot()
#from graphviz import Source
#src = Source(dot)
#src

## Expression simplification

The expression simplification applies transformation rules to an expression until they can be applied. This process is done by an `ExpressionSimplifier` object, implemented in `miasm.expression.simplifications`.

Some basic simplifications are already implemented and can be activated with the `expr_simp` instance in the module.

In [23]:
from miasm.expression.simplifications import expr_simp

# 0x10 + (-1) = 0xF
op3 = cst1 + cst2
print(op3)
cst3 = expr_simp(op3)
print(cst3)

# 5th bit of 0x10 = 1
sl2 = cst1[4:5]
print(sl2)
cst4 = expr_simp(sl2)
print(cst4)

# a + a - a = a
op4 = a + a - a
print(op4)
print(expr_simp(op4))
assert expr_simp(op4) == a

# Use to evaluate an expression (here a + 0x10 is evaluated with a = 0x10)
print(op1)
print(op1.replace_expr({a: cst1}))
print(expr_simp(op1.replace_expr({a: cst1})))
print(expr_simp(a + a +a + a))

0x10 + 0xFFFFFFFF
0xF
0x10[4:5]
0x1
a + a + -a
a
a + 0x10
0x10 + 0x10
0x20
a * 0x4


Transformation rules can be added with the method `enable_passes`. A transformation rule is a function and is associated to an expression type.

Below, the code transforms the booleans operation in an `ExprCond` to an operation of type `<`.

In [24]:
x = ExprId("x", 32)
y = ExprId("y", 32)

inf_signed = ((x - y) ^ ((x ^ y) & ((x - y) ^ x)))[31:32]
print(inf_signed)

def is_inf(x_val, y_val):
    new_val = expr_simp(inf_signed.replace_expr({
        x: x_val,
        y: y_val,
    }))
    assert new_val.is_int()
    return int(new_val) == 1

# 0 < 10
print(is_inf(ExprInt(0, 32), ExprInt(10, 32)))
# 10 !< 10
print(is_inf(ExprInt(10, 32), ExprInt(10, 32)))
# -1 < 0
print(is_inf(ExprInt(0xFFFFFFFF, 32), ExprInt(0, 32)))

((x + -y) ^ ((x ^ y) & ((x + -y) ^ x)))[31:32]
True
False
True


The following code enables this transformation, which was already implemented but not enabled: 

In [25]:
from pprint import pprint as pp
from miasm.expression.simplifications import ExpressionSimplifier
pp(ExpressionSimplifier.PASS_COND)

print(expr_simp(inf_signed))
expr_simp_cond = ExpressionSimplifier()
expr_simp_cond.enable_passes(ExpressionSimplifier.PASS_COND)
print(expr_simp_cond(inf_signed))

{<class 'miasm.expression.expression.ExprCond'>: [<function expr_simp_equal at 0x7f9af40ce290>],
 <class 'miasm.expression.expression.ExprOp'>: [<function expr_simp_inverse at 0x7f9af40ce200>],
 <class 'miasm.expression.expression.ExprSlice'>: [<function expr_simp_inf_signed at 0x7f9af40ce0e0>,
                                                   <function expr_simp_inf_unsigned_inversed at 0x7f9af40ce170>]}
(((x ^ y) & (x ^ (x + -y))) ^ (x + -y))[31:32]
x <s y


### Exercice 1 : Add a new transformation rule

The current goal is to create a new simplification rule. The rule is informally the following:

*A left shift of *n* bits followed by a right shift of *size - n* bits is equivalent to a rotation to the left of *n* bits.*

For example, `a << 14 | a >> 18` becomes `a <<< 14` (if `a` is on 32 bits and `<<<` is the left rotation operator).

In Miasm, a transformation rule is actually a function whose two parameters are:
* An `ExpressionSimplifier` instance used for applying simplifications ;
* An `Expr` to simplify

The function **must always return an Expr**. If nothing has changed, the function will return its second argument.

A transformation rule must generate a *new* expression. Indeed, the Miasm expressions are immutables, so if a modification occurs, a new instance must be created to include the modifications.

In [26]:
a = ExprId("a", 32)
b = ExprId("b", 32)
cst1 = ExprInt(16, 32)

# Tests vectors
tests = [
    # (entrée, sortie attendue)
    (a, a),
    (a << ExprInt(4, 32), (a << ExprInt(4, 32))),
    ((a << ExprInt(4, 32) | a >> ExprInt(28, 32)), ExprOp("<<<", a, ExprInt(4, 32))),
    ((a << ExprInt(4, 32) | a >> ExprInt(27, 32)), (a << ExprInt(4, 32) | a >> ExprInt(27, 32))),
    ((a >> ExprInt(28, 32) | a << ExprInt(4, 32)), ExprOp("<<<", a, ExprInt(4, 32))),
]

# Vérification
def check(tests, custom_expr_simp):
    for inp, out in tests:
        print(inp)
        got = custom_expr_simp(inp)
        print(got)
        assert out.canonize() == got.canonize()

from collections import Counter

def mysimplification(simp_engine, expr):
    # TODO
    return expr

from miasm.expression.simplifications import ExpressionSimplifier
simp_engine = ExpressionSimplifier()

simp_engine.enable_passes({
    ExprOp: [mysimplification]
})

# Tests launch
check(tests, simp_engine)



a
a
a << 0x4
a << 0x4
(a << 0x4) | (a >> 0x1C)
(a << 0x4) | (a >> 0x1C)


AssertionError: 

In [ ]:
# Add a more complex test case

tests += [
    ((a << ExprInt(4, 32)) | a >> (ExprInt(32, 32) - ExprInt(4, 32)), ExprOp("<<<", a, ExprInt(4, 32))),
]

# TODO: Activate constant propagation passes

# Tests launch
check(tests, simp_engine)

## Expression matching

Miasm embeds a tiny regular expression engine, called `match_expr`, in order to ease the writing of transformation rules.

Its parameters are:
* The expression to analyze;
* An expression describing the pattern to match;
* A list of *jokers*, i.e. the expressions that can be replaced inside the *pattern*

For example, if we want to match the expression `X + (X * Y) + EAX`, where `X` and `Y` are placeholders, the `match_expr`instanciation will look like:

`match_expr(expr_to_match, X + (X * Y) + EAX, [X, Y])`.

The results is a dictionnary associating each joker with the matching subexpressions found. If the operator is commutative, the order can change.

### Exercice 2 : expression matching

Implement the following simplification rule using the match_expr: `((x & y) + (x | y))` -> `(x + y)`

In [ ]:
from miasm.expression.expression import match_expr
x = ExprId("x", 32)
y = ExprId("y", 32)
# tests vectors
tests = [
    (((x & y) + (x | y)), (x + y)),
    (((x & y) + (x & y)), ((x & y) + (x & y))),
    (((cst1 >> a) | (a ^ cst1)) + (((a ^ cst1) & (cst1 >> a))), (cst1 >> a) + (a ^ cst1)),
]

# jokers
X = ExprId("X", 32)
Y = ExprId("Y", 32)

def my(e_s, expr):
    # TODO: transformation rule
    res = match_expr(...)
    if res:
        return res[X] + res[Y]
    return expr

simp_engine = ExpressionSimplifier()
simp_engine.enable_passes({ExprOp:[my]})

check(tests, simp_engine)

## Going further

Some functionalities relative to expressions but not detailed here:
* `SymbolicExecutionEngine` : Symbolic execution;
* `Translators` : expression translation to C, Python, "Miasm like" or z3 expressions;
* `expr_range` : Range analysis of an expression possible values;
* `AssignBlock`, `IRBlock`, `DiGraphDefUse`, `dead_simp`, ... : grouping expressions to describe a full program, associated analysis;
* `miasm.arch.*.sem.py`, `SemBuilder` : architecture specific semantic descriptions, i.e. all the side effects of a mnemonic.